In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

In [65]:
basics_df = pd.read_csv("https://datasets.imdbws.com/title.basics.tsv.gz", sep="\t", low_memory=False)

In [ ]:
basics_df =  basics_df.drop('endYear',axis=1)

In [74]:
movies = basics_df.loc[basics_df['titleType']=='movie']

In [75]:
movies = movies.drop('titleType',axis=1)

In [ ]:
movies_ok = movies.loc[movies['isAdult']=='0']
movies_ok=movies_ok.drop('isAdult',axis=1)

In [78]:
movies_ok_nat = movies_ok.replace('\\N', pd.NaT)
movies_clean = movies_ok_nat.dropna()

In [79]:
movies_clean = movies_clean.astype({'runtimeMinutes': int})
movies_clean = movies_clean.astype({'startYear': int})

In [ ]:
movies_clean = movies_clean[(movies_clean['runtimeMinutes'] >= 58) & (movies_clean['runtimeMinutes'] <= 270)]

In [ ]:
movies_clean = movies_clean[(movies_clean['startYear'] >= 1918) & (movies_clean['startYear'] <= 2021)]

In [82]:
ratings_df = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz", sep="\t")

In [88]:
movies = pd.merge(movies_clean,ratings_df,how='inner')

In [89]:
movies_7 = movies.loc[movies['averageRating']>=7.0]

In [90]:
movies_7_rating = movies_7[movies_7['numVotes'] >= 1000]


In [ ]:
crew_df = pd.read_csv("https://datasets.imdbws.com/title.crew.tsv.gz", sep="\t")

In [19]:
principals_df = pd.read_csv("https://datasets.imdbws.com/title.principals.tsv.gz", sep="\t")

In [20]:
name_df = pd.read_csv("https://datasets.imdbws.com/name.basics.tsv.gz", sep="\t")

In [128]:
movie_dir = pd.merge(movies_7_rating,crew_df,how='inner',on='tconst')

In [129]:
movie_dir.drop('writers',axis=1,inplace=True)

In [130]:
movie_dir.drop('primaryTitle',axis=1,inplace=True)

In [131]:
movie_dir[['mainDirector', 'director2']] = movie_dir['directors'].str.split(',', n=1, expand=True)

In [132]:
movie_dir.drop('directors',axis=1,inplace=True)
movie_dir.drop('director2',axis=1,inplace=True)

In [133]:
movie_dir = movie_dir.replace('\\N', pd.NaT)

In [138]:
movie_dir.dropna(subset=['mainDirector'], inplace=True)


In [139]:
movie_dir = pd.concat([movie_dir[['tconst', 'originalTitle', 'startYear', 'runtimeMinutes','genres','averageRating','numVotes']] , movie_dir['mainDirector'].str.get_dummies()], 
          axis = 1)

In [140]:
movie_dir[['genre1', 'genre2','genre3']] = movie_dir['genres'].str.split(',', n=2, expand=True)

In [141]:
movie_dir = pd.concat([movie_dir , movie_dir['genre1'].str.get_dummies(), movie_dir['genre2'].str.get_dummies(), movie_dir['genre3'].str.get_dummies()], 
          axis = 1)

In [144]:
movie_dir.drop('genres',axis=1,inplace=True)

In [145]:
movie_dir

,tconst,originalTitle,startYear,runtimeMinutes,averageRating,numVotes,nm0000005,nm0000008,nm0000019,nm0000033,...,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0008879,Berg-Ejvind och hans hustru,1918,136,7.2,1784,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,tt0009893,Die Austernprinzessin,1919,60,7.2,1817,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,tt0009937,Blind Husbands,1919,99,7.0,1470,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,1919,90,7.3,10030,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,tt0010247,Herr Arnes pengar,1919,122,7.2,1356,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11638,tt9882084,Chasing Happiness,2019,96,7.8,2138,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
11639,tt9886872,Munthiri Monchan,2019,130,7.8,1345,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11640,tt9900782,Kaithi,2019,145,8.5,18842,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
11641,tt9902160,Herself,2020,97,7.0,2850,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
